<a href="https://colab.research.google.com/github/FaddliLWibowo/Twitter-Sentiment-Analysis-with-Streamlit/blob/main/Twitter_Sentiment_Analysis_Model_with_Tuning_Hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PySastrawi==1.2.0
!pip install snscrape==0.5.0.20230113
!pip install nltk==3.7
!pip install pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import some libraries
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
seed = 0
np.random.seed(seed)
import datetime as dt
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style = 'whitegrid')
import pickle


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud


import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM, SpatialDropout1D
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score
# from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
tweets = pd.read_excel("https://github.com/FaddliLWibowo/Twitter-Sentiment-Analysis-with-Streamlit/blob/main/Dataset/Data-hasil-klasifikasi-positif-negatif.xlsx?raw=true")
tweets = tweets[['text']]
tweets.columns = ['tweet']
tweets.sample(5)

,tweet
16313,opini mengenai kuliah online positif dan negat...
10847,kalau dia bertengkar tidak ada telur busuk dik...
18,oh jadi ikn tidak selesai yha…
20471,kuliah daring gak makin rajin malah makin mala...
28516,lu padahal lg kuliah online tp disangka leha i...


## Preprocessing Text Data

In [4]:
# Some functions for preprocessing text
def cleaningText(tweet):
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet) # remove mentions
    tweet = re.sub(r'#[A-Za-z0-9]+', '', tweet) # remove hashtag
    tweet = re.sub(r'RT[\s]', '', tweet) # remove RT
    tweet = re.sub(r"http\S+", '', tweet) # remove link
    tweet = re.sub(r'[0-9]+', '', tweet) # remove numbers
    tweet = tweet.replace('\n', ' ') # replace new line into space
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    tweet = tweet.strip(' ') # remove characters space from both left and right text
    return tweet

def casefoldingText(tweet): # Converting all the characters in a text into lower case
    tweet = tweet.lower() 
    return tweet

key_norm = pd.read_csv('https://raw.githubusercontent.com/FaddliLWibowo/Twitter-Sentiment-Analysis-with-Streamlit/main/Dataset/kamus-slang-ind.csv', encoding='ISO-8859-1')
def text_normalize(tweet):
    tweet = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0]
    if (key_norm['singkat'] == word).any()
    else word for word in tweet.split()
    ])
    return tweet

def tokenizingText(tweet): # Tokenizing or splitting a string, text into a list of tokens
    tweet = word_tokenize(tweet) 
    return tweet

def filteringText(tweet): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in tweet:
        if txt not in listStopwords:
            filtered.append(txt)
    tweet = filtered 
    return tweet

def stemmingText(tweet): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tweet = [stemmer.stem(word) for word in tweet]
    return tweet

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

In [5]:
# Preprocessing tweets data
tweets['text_clean'] = tweets['tweet'].apply(cleaningText)
tweets['text_clean'] = tweets['text_clean'].apply(casefoldingText)
tweets['text_clean'] = tweets['text_clean'].apply(text_normalize)
tweets.drop(['tweet'], axis = 1, inplace = True)

tweets['text_preprocessed'] = tweets['text_clean'].apply(tokenizingText)
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(filteringText)
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(stemmingText)
tweets.drop_duplicates(subset = 'text_clean', inplace = True)

In [6]:
# Determine sentiment polarity of tweets using indonesia sentiment lexicon (source : https://github.com/fajri91/InSet)
lexicon_positive = {}
positive = 'https://raw.githubusercontent.com/FaddliLWibowo/Twitter-Sentiment-Analysis-with-Streamlit/main/Dataset/lexicon_positive.csv'
positive  = pd.read_csv(positive )
for index, row in positive.iterrows():
    lexicon_positive[row[0]] = int(row[1])

lexicon_negative = {}
negative = 'https://raw.githubusercontent.com/FaddliLWibowo/Twitter-Sentiment-Analysis-with-Streamlit/main/Dataset/lexicon_negative.csv'
negative  = pd.read_csv(negative )
for index, row in negative.iterrows():
    lexicon_negative[row[0]] = int(row[1])
               
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
    score = 0
    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
    polarity=''
    if (score > 0):
        polarity = 'positif'
    elif (score < 0):
        polarity = 'negatif'
    else:
        polarity = 'positif' # dapat diganti dengan netral
    return score, polarity

In [7]:
# Results from determine sentiment polarity of tweets
results = tweets['text_preprocessed'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
tweets['polarity_score'] = results[0]
tweets['polarity'] = results[1]

In [8]:
# Make text preprocessed (tokenized) to untokenized with toSentence Function
X = tweets['text_preprocessed'].apply(toSentence) 
max_features = 5000

# Tokenize text with specific maximum number of words to keep, based on word frequency
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X.values)
print("Index: "+str(tokenizer.word_index))
X = tokenizer.texts_to_sequences(X.values)
X = pad_sequences(X)
X.shape

Index: {'kuliah': 1, 'online': 2, 'ikn': 3, 'ya': 4, 'banget': 5, 'bangun': 6, 'bola': 7, 'sepak': 8, 'tugas': 9, 'nusantara': 10, 'indonesia': 11, 'sih': 12, 'of': 13, 'nya': 14, 'international': 15, 'federation': 16, 'association': 17, 'pssi': 18, 'orang': 19, 'pakai': 20, 'rumah': 21, 'dosen': 22, 'pas': 23, 'presiden': 24, 'kerja': 25, 'offline': 26, 'jam': 27, 'bikin': 28, 'nih': 29, 'gara': 30, 'tau': 31, 'jokowi': 32, 'wkwk': 33, 'teman': 34, 'semangat': 35, 'negara': 36, 'semester': 37, 'moga': 38, 'pagi': 39, 'tidur': 40, 'dukung': 41, 'kota': 42, 'masuk': 43, 'mahasiswa': 44, 'ajar': 45, 'anak': 46, 'kasih': 47, 'kelas': 48, 'biar': 49, 'suka': 50, 'polisi': 51, 'enak': 52, 'kuota': 53, 'iya': 54, 'pilih': 55, 'kampus': 56, 'jalan': 57, 'handphone': 58, 'erick': 59, 'beli': 60, 'pindah': 61, 'habis': 62, 'ku': 63, 'langsung': 64, 'malas': 65, 'laptop': 66, 'materi': 67, 'selesai': 68, 'cepat': 69, 'salah': 70, 'mata': 71, 'butuh': 72, 'jakarta': 73, 'cape': 74, 'nasional': 75

(36562, 82)

In [9]:
import pickle
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# Encode target data into numerical values
polarity_encode = {'negatif' : 0, 'positif' : 1}
y = tweets['polarity'].map(polarity_encode).values

# Split the data (with composition data train 80%, data test 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(29249, 82) (29249,)
(7313, 82) (7313,)


## Model LSTM

In [11]:
# Create model function with default hyperparameter values
def create_model(embed_dim = 32, hidden_unit = 32, dropout_rate = 0.2, optimizers = Adam, learning_rate = 0.001):
    model = Sequential()
    model.add(Embedding(input_dim = max_features, output_dim = embed_dim, input_length = X_train.shape[1]))
    model.add(LSTM(units = hidden_unit, activation = 'tanh'))
    model.add(Dropout(dropout_rate))
    # model.add(Dense(units = 2, activation = 'softmax'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    # model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizers(lr = learning_rate), metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer = optimizers(learning_rate = learning_rate), metrics = ['accuracy'])
    print(model.summary())
    return model

In [ ]:
# Hyperparameter tuning (to know the best hyperparameter for this model)

# Wrapper model with default hyperparameter values
model = KerasClassifier(build_fn = create_model, epochs = 20, batch_size=128) 

# Hyperparameters
embed_dim = [64]
hidden_unit = [32, 64]
dropout_rate = [0.2]
optimizers = [RMSprop]
learning_rate = [0.001]
epochs = [10]
batch_size = [128]
param_grid = dict(embed_dim = embed_dim, hidden_unit = hidden_unit, dropout_rate = dropout_rate,
                  learning_rate = learning_rate, optimizers = optimizers, epochs = epochs, batch_size = batch_size)

# Evaluation model with GridSearchCV to know what the best hyperparameter for model 
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3)
grid_result = grid.fit(X_train, y_train)

results = pd.DataFrame()
results['means'] = grid_result.cv_results_['mean_test_score']
results['stds'] = grid_result.cv_results_['std_test_score']
results['params'] = grid_result.cv_results_['params']
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
results.to_csv(r'gridsearchcv_results.csv', index = False, header = True)
results.sort_values(by='means', ascending = False).reset_index(drop=True)

<ipython-input-12-814bb67e8b7c>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = create_model, epochs = 20, batch_size=128)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 82, 64)            320000    
                                                                 
 lstm (LSTM)                 (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 332,449
Trainable params: 332,449
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
153/153 [==============================] - 19s 88ms/step - loss: 0.5083 - accuracy: 0.7540
Epoch 2/10
153/153 [==============================] - 13s 82ms/step - loss:

In [ ]:
# Results from hyperparameter tuning
results = pd.read_csv('gridsearchcv_results.csv')
results.sort_values(by='means', ascending = False).reset_index(drop=True)

In [ ]:
# From results above, we know the best hyperparameter for this model is :
# {'batch_size': 128, 'dropout_rate': 0.2, 'embed_dim': 32, 'epochs': 10, 'hidden_unit': 16, 'learning_rate': 0.001, 'optimizers': <class 'keras.optimizers.RMSprop'>}

# Create the model with the best hyperparameter which has been determined
model = KerasClassifier(build_fn = create_model,
                        # Model Parameters
                        dropout_rate = 0.2,
                        embed_dim = 32,
                        hidden_unit = 32,
                        optimizers = RMSprop,
                        learning_rate = 0.001,
                        # Fit Parameters
                        epochs=20, 
                        batch_size=128,
                        # Initiate validation data, which is 10% data from data train. It's used for evaluation model
                        validation_split = 0.2)
                         
model_prediction = model.fit(X_train, y_train)

In [ ]:
# Visualization model accuracy (train and val accuracy)

fig, ax = plt.subplots(figsize = (10, 4))
ax.plot(model_prediction.history['accuracy'], label = 'train accuracy')
ax.plot(model_prediction.history['val_accuracy'], label = 'val accuracy')
ax.set_title('Model Accuracy')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.legend(loc = 'upper left')
plt.show()

In [ ]:
# Predict sentiment on data test by using model has been created, and then visualize a confusion matrix
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Model Accuracy on Test Data:', accuracy)
confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize = (8,6))
sns.heatmap(confusion_matrix(y_true = y_test, y_pred = y_pred), fmt = 'g', annot = True)
ax.xaxis.set_label_position('top')
ax.xaxis.set_ticks_position('top')
ax.set_xlabel('Prediction', fontsize = 14)
ax.set_xticklabels(['negative (0)', 'positive (1)'])
ax.set_ylabel('Actual', fontsize = 14)
ax.set_yticklabels(['negative (0)', 'positive (1)'])
plt.show()

## Predict with Other Data

In [ ]:
# Initializing and preprocessing new text data
otherData = pd.DataFrame()
otherData['text'] = ['enaknya kuliah online,  ga perlu capek" ke kampus dan bisa santai dirumah',
                     'tidak berguna' 
                    ]

otherData['text_clean'] = otherData['text'].apply(cleaningText)
otherData['text_clean'] = otherData['text_clean'].apply(casefoldingText)
otherData.drop(['text'], axis = 1, inplace = True)

otherData['text_preprocessed'] = otherData['text_clean'].apply(tokenizingText)
otherData['text_preprocessed'] = otherData['text_preprocessed'].apply(filteringText)
otherData['text_preprocessed'] = otherData['text_preprocessed'].apply(stemmingText)
otherData

In [ ]:
# Preprocessing text data

# Make text preprocessed (tokenized) to untokenized with toSentence Function
X_otherData = otherData['text_preprocessed'].apply(toSentence)
X_otherData = tokenizer.texts_to_sequences(X_otherData.values)
X_otherData = pad_sequences(X_otherData, maxlen = X.shape[1])
X_otherData

In [ ]:
# Results from prediction sentiment on text data

y_pred_otherData = model.predict(X_otherData)
otherData['Result Prediction'] = y_pred_otherData

polarity_decode = {0 : 'negatif', 1 : 'positif'}
otherData['Result Prediction'] = otherData['Result Prediction'].map(polarity_decode)
otherData